<a href="https://colab.research.google.com/github/rajeevperli666/banking-system-using-python/blob/main/Blockchain_Voting_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (only needed once)
!pip install gradio pandas matplotlib plotly

import gradio as gr
import pandas as pd
import hashlib
import time
import tempfile
import matplotlib.pyplot as plt
import plotly.express as px
from io import BytesIO
from PIL import Image

# === Blockchain Initialization ===
blockchain = []

# === Candidate List ===
candidates = ["Alice", "Bob", "Charlie"]

# === Create a new block ===
def create_block(voter_id, candidate):
    previous_hash = blockchain[-1]['hash'] if blockchain else "0"
    timestamp = str(time.time())

    block_string = f"{voter_id}-{candidate}-{timestamp}-{previous_hash}"
    block_hash = hashlib.sha256(block_string.encode()).hexdigest()

    block = {
        "voter_id_hash": hashlib.sha256(voter_id.encode()).hexdigest(),
        "candidate": candidate,
        "timestamp": timestamp,
        "previous_hash": previous_hash,
        "hash": block_hash,
    }

    blockchain.append(block)
    return f"✅ Vote successfully cast for {candidate}!"

# === Check if a voter has already voted ===
def has_already_voted(voter_id):
    voter_hash = hashlib.sha256(voter_id.encode()).hexdigest()
    return any(block['voter_id_hash'] == voter_hash for block in blockchain)

# === Cast vote handler ===
def vote(voter_id, candidate):
    if not voter_id or not candidate:
        return "⚠️ Please provide both Voter ID and select a candidate.", None
    if has_already_voted(voter_id):
        return f"❌ Voter ID '{voter_id}' has already voted.", None

    msg = create_block(voter_id, candidate)
    return msg, get_results()

# === Get live vote results ===
def get_results():
    df = pd.DataFrame(blockchain)
    if df.empty:
        return pd.DataFrame(columns=["Candidate", "Votes"])
    results = df['candidate'].value_counts().reset_index()
    results.columns = ['Candidate', 'Votes']
    return results

# === Export blockchain to CSV with totals ===
def export_chain():
    df = pd.DataFrame(blockchain)
    vote_summary = df['candidate'].value_counts().reset_index()
    vote_summary.columns = ['Candidate', 'Total Votes']
    separator = pd.DataFrame([['---'] * 5], columns=df.columns)
    blank = pd.DataFrame([[''] * 5], columns=df.columns)
    combined = pd.concat([df, separator, blank], ignore_index=True)
    summary_block = vote_summary.rename(columns={
        'Candidate': 'voter_id_hash',
        'Total Votes': 'candidate'
    })
    summary_block['timestamp'] = ''
    summary_block['previous_hash'] = ''
    summary_block['hash'] = ''
    final_df = pd.concat([combined, summary_block], ignore_index=True)
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".csv", mode='w', newline='')
    final_df.to_csv(temp.name, index=False)
    temp.flush()
    return temp.name

# === Pie Chart for Live Results ===
def plot_pie_chart():
    df = pd.DataFrame(blockchain)
    if df.empty:
        return None
    pie = px.pie(df, names='candidate', title='Live Vote Distribution', hole=0.4)
    return pie

# === Bar Chart for Live Results ===
def plot_bar_chart():
    df = pd.DataFrame(blockchain)
    if df.empty:
        return None
    count_data = df['candidate'].value_counts().reset_index()
    count_data.columns = ['Candidate', 'Votes']
    fig = px.bar(count_data, x='Candidate', y='Votes', title='Vote Count by Candidate',
                 color='Candidate', text='Votes')
    return fig

# === Vote Timeline Chart (Matplotlib with PIL fix) ===
def plot_timeline():
    df = pd.DataFrame(blockchain)
    if df.empty:
        return None
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df['count'] = 1
    df = df.set_index('timestamp').resample('1Min').sum().fillna(0)
    fig, ax = plt.subplots(figsize=(8, 4))
    df['count'].plot(ax=ax, color='blue', marker='o')
    ax.set_title("Vote Frequency Over Time (Votes per Minute)")
    ax.set_ylabel("Number of Votes")
    ax.set_xlabel("Time")
    plt.tight_layout()
    buf = BytesIO()
    fig.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    return img

# === Fraud Detection Logic (Detects Vote Spikes) ===
def detect_fraud():
    df = pd.DataFrame(blockchain)
    if df.empty:
        return "No data to analyze."
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df['minute'] = df['timestamp'].dt.floor('Min')
    spike = df['minute'].value_counts().sort_values(ascending=False)
    if spike.empty:
        return "No suspicious voting patterns detected."
    threshold = spike.mean() + 3 * spike.std()
    suspicious = spike[spike > threshold]
    if not suspicious.empty:
        spike_times = ', '.join(str(ts) for ts in suspicious.index)
        return (f"⚠️ Fraud Alert: Unusual spikes detected at these times:\n"
                f"{spike_times}\nWith votes counts: {list(suspicious.values)}")
    return "✅ No suspicious voting patterns detected."

# === Admin view of raw blockchain data ===
def admin_view():
    return pd.DataFrame(blockchain)

# === Gradio UI ===
with gr.Blocks() as app:
    gr.Markdown("## 🗳️ VoteChain – Blockchain Voting System")
    gr.Markdown("Secure voting with live analytics, fraud detection, and CSV export.")

    with gr.Row():
        voter_input = gr.Textbox(label="🆔 Enter Voter ID", placeholder="e.g., VOTER123")
        candidate_input = gr.Radio(candidates, label="👤 Choose Candidate")

    vote_button = gr.Button("✅ Cast Vote")
    vote_output = gr.Textbox(label="System Message", interactive=False)
    results_output = gr.Dataframe(label="📊 Live Results")

    vote_button.click(vote, inputs=[voter_input, candidate_input], outputs=[vote_output, results_output])

    gr.Markdown("### 📥 Download Full Blockchain Data")
    export_btn = gr.Button("📁 Export as CSV")
    csv_file = gr.File(label="Blockchain CSV Export")
    export_btn.click(export_chain, outputs=csv_file)

    gr.Markdown("### 📊 Visual Insights")
    pie_chart_btn = gr.Button("Show Pie Chart")
    pie_output = gr.Plot()
    pie_chart_btn.click(plot_pie_chart, outputs=pie_output)

    bar_chart_btn = gr.Button("Show Bar Chart")
    bar_output = gr.Plot()
    bar_chart_btn.click(plot_bar_chart, outputs=bar_output)

    timeline_btn = gr.Button("📅 Vote Timeline Chart")
    timeline_output = gr.Image()
    timeline_btn.click(plot_timeline, outputs=timeline_output)

    gr.Markdown("### 🧠 Vote Fraud Detection")
    fraud_btn = gr.Button("Run Fraud Analysis")
    fraud_output = gr.Textbox(label="🛡️ Fraud Detection Report")
    fraud_btn.click(detect_fraud, outputs=fraud_output)

    gr.Markdown("### 🔐 Admin View – Blockchain Ledger")
    admin_btn = gr.Button("View Blockchain Ledger")
    admin_output = gr.Dataframe()
    admin_btn.click(admin_view, outputs=admin_output)

app.launch()